In [1]:
%cd ../

/home/qwj/code/HippoRAG


In [2]:
import pandas as pd

## Statistics of Triplets

In [20]:
triples_df = pd.read_csv("output/musique_queries.triples.tsv", sep='\t')
triples_df.head()

,Unnamed: 0.1,Unnamed: 0,id,question,triples,extracted_triples
0,0,0,2hop__13548_13529,When was the person who Messi's goals in Copa ...,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""...","{'triplets': [['#1', 'compared by', 'Messi'], ..."
1,1,1,3hop1__9285_5188_23307,What month did the Tripartite discussions begi...,"{""named_entities"": [""Britain"", ""France"", ""Wars...","{'triplets': [['the country where', 'the top-r..."
2,2,2,2hop__766973_770570,What county is Erik Hort's birthplace a part of?,"{""named_entities"": [""Erik Hort""]}","{'triplets': [['Erik Hort', 'birthplace', '#1'..."
3,3,3,2hop__170823_120171,What year did the publisher of Labyrinth end?,"{""named_entities"": [""Labyrinth""]}","{'triplets': [['Labyrinth', 'published by', '#..."
4,4,4,2hop__511454_120259,When was Lady Godiva's birthplace abolished?,"{""named_entities"": [""Lady Godiva's birthplace""]}","{'triplets': [[""Lady Godiva's birthplace"", 'ab..."


In [28]:
triples_df.head(10)

,Unnamed: 0.1,Unnamed: 0,id,question,triples,extracted_triples
0,0,0,2hop__13548_13529,When was the person who Messi's goals in Copa ...,"{""named_entities"": [""Messi"", ""Copa del Rey"", ""...","[[#1, compared by, Messi], [Messi, scored for,..."
1,1,1,3hop1__9285_5188_23307,What month did the Tripartite discussions begi...,"{""named_entities"": [""Britain"", ""France"", ""Wars...","[[the country where, the top-ranking Warsaw Pa..."
2,2,2,2hop__766973_770570,What county is Erik Hort's birthplace a part of?,"{""named_entities"": [""Erik Hort""]}","[[Erik Hort, birthplace, #1], [#1, part of, co..."
3,3,3,2hop__170823_120171,What year did the publisher of Labyrinth end?,"{""named_entities"": [""Labyrinth""]}","[[Labyrinth, published by, #1], [#1, ended, #2]]"
4,4,4,2hop__511454_120259,When was Lady Godiva's birthplace abolished?,"{""named_entities"": [""Lady Godiva's birthplace""]}","[[Lady Godiva's birthplace, abolished, #1]]"
5,5,5,4hop2__71753_648517_70784_79935,When was the region immediately north of the r...,"{""named_entities"": [""Israel"", ""Battle of Qurah...","[[#1, is located in, region north of], [#2, to..."
6,6,6,4hop1__94201_642284_131926_89261,Where does the body of water by the city where...,"{""named_entities"": [""Southeast Library"", ""Gulf...",[[body of water by the city where the Southeas...
7,7,7,4hop1__152562_5274_458768_33633,When did the explorer reach the city where the...,"{""named_entities"": [""Vilaiyaadu Mankatha""]}","[[#1, reached, #2], [#2, located in city where..."
8,8,8,3hop1__305282_282081_73772,When was the start of the battle of the birthp...,"{""named_entities"": [""battle of the birthplace""...","[[performer of III, performer in, #1 (battle)]..."
9,9,9,2hop__655505_110949,What is the Till dom ensamma performer's birth...,"{""named_entities"": [""Till dom ensamma""]}","[[Till dom ensamma, has birth date, #1]]"


In [24]:
import json

def convert_string_to_json(json_str):
    return eval(json_str)['triplets']

n_errors = 0
for i, row in triples_df.iterrows():
    try:
        convert_string_to_json(row['extracted_triples'])
    except:
        n_errors += 1
        # print(row['question'], row['extracted_triples'])
n_errors

87

In [25]:
def convert_string_to_json_v2(json_str):
    try:
        return eval(json_str)['triplets']
    except:
        return None

triples_df['extracted_triples'] = triples_df['extracted_triples'].apply(convert_string_to_json_v2)

In [27]:
triples_df['extracted_triples'].tolist()

[[['#1', 'compared by', 'Messi'],
  ['Messi', 'scored for', 'Copa del Rey'],
  ['#1', 'signed by', 'Barcelona']],
 [['the country where',
   'the top-ranking Warsaw Pact operatives originated',
   '#1'],
  ['#1', 'headquartered in', 'the nobilities commonwealth'],
  ['#1', 'discussions begin', 'month #2'],
  ['Britain', 'discussions begin', 'month #3'],
  ['France', 'discussions begin', 'month #4']],
 [['Erik Hort', 'birthplace', '#1'], ['#1', 'part of', 'county']],
 [['Labyrinth', 'published by', '#1'], ['#1', 'ended', '#2']],
 [["Lady Godiva's birthplace", 'abolished', '#1']],
 [['#1', 'is located in', 'region north of'],
  ['#2', 'took place in', '#3'],
  ['#3', 'created', '#4']],
 [['body of water by the city where the Southeast Library designer died',
   'empties into',
   'Gulf of Mexico'],
  ['Southeast Library designer', 'died in', '#1'],
  ['#1',
   'is located at the city where',
   'body of water by the city where the Southeast Library designer died']],
 [['#1', 'reached', '

In [29]:
from collections import defaultdict, deque

def triples_to_chain(triples):
    # Step 1: Extract entities and relationships
    entities = set()
    for triple in triples:
        entities.add(triple[0])
        entities.add(triple[2])

    # Step 2: Build the graph using adjacency list
    graph = defaultdict(list)
    for triple in triples:
        graph[triple[0]].append((triple[1], triple[2]))
    
    # Step 3: Identify paths (chains) using BFS or DFS
    chains = []
    def dfs(node, path):
        if not graph[node]:
            chains.append(path)
            return
        for relation, neighbor in graph[node]:
            dfs(neighbor, path + [(node, relation, neighbor)])

    # Identify all start nodes (nodes that are not destination in any triple)
    start_nodes = {triple[0] for triple in triples} - {triple[2] for triple in triples}
    
    # Start DFS from all start nodes
    for start in start_nodes:
        dfs(start, [])

    return chains

# Example usage
triples = [
    ['the country where', 'the top-ranking Warsaw Pact operatives originated', '#1'],
    ['#1', 'headquartered in', 'the nobilities commonwealth'],
    ['#1', 'discussions begin', 'month #2'],
    ['Britain', 'discussions begin', 'month #3'],
    ['France', 'discussions begin', 'month #4']
]

chains = triples_to_chain(triples)
for chain in chains:
    print(chain)


[('Britain', 'discussions begin', 'month #3')]
[('the country where', 'the top-ranking Warsaw Pact operatives originated', '#1'), ('#1', 'headquartered in', 'the nobilities commonwealth')]
[('the country where', 'the top-ranking Warsaw Pact operatives originated', '#1'), ('#1', 'discussions begin', 'month #2')]
[('France', 'discussions begin', 'month #4')]


### Hipporag

In [2]:
from src.hipporag_v3 import HipporagConfig, HippoRAGv3

config = HipporagConfig(
    corpus_name='musique',
    extraction_model='openai',
    extraction_model_name='gpt-3.5-turbo-1106',
    graph_creating_retriever_name='colbertv2',
    sim_threshold=0.8,
    damping=0.5,
)

/home/qwj/miniconda3/envs/hipporag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

hipporag = HippoRAGv3(config)

Augmenting Graph from Similarity


Building Graph: 100%|██████████| 394346/394346 [00:00<00:00, 598403.93it/s]
2024-07-26 09:04:02,633 - src.hipporag_v3 - INFO - Graph built: num vertices: 91729, num_edges: 394235




[Jul 26, 09:04:13] #> Note: Output directory data/lm_vectors/colbert/musique/phrase/indexes/nbits_2 already exists


[Jul 26, 09:04:23] #> Loading collection...
0M 
[Jul 26, 09:04:34] #> Loading codec...
[Jul 26, 09:04:34] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 26, 09:04:34] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 26, 09:04:35] #> Loading IVF...
[Jul 26, 09:04:35] #> Loading doclens...


100%|██████████| 4/4 [00:00<00:00, 1066.91it/s]

[Jul 26, 09:04:35] #> Loading codes and residuals...



100%|██████████| 4/4 [00:00<00:00, 297.49it/s]


In [5]:
len(hipporag.similar_relations)

35512

In [7]:
hipporag.similar_relations[10]

{4, 7921}